In [1]:
#Import statements

import numpy as np
import math as math
import sys
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from autils import *
%matplotlib inline
import pandas as pd
np.set_printoptions(threshold=sys.maxsize)

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [3]:
#Reading all of the csv files. The csv files are stored as pandas dataframes, which are then converted to numpy arrays
#During the conversion process, the first row of the data is essentially lost; keep this in mind when trying to match inputs/outputs to the csv
df1 = pd.read_csv('input_data.csv')
inputs = df1.to_numpy()

df2 = pd.read_csv('output_data.csv')
outputs = df2.to_numpy()

print ('The shape of the input training is: ' + str(inputs.shape))
print ('The shape of the output training is: ' + str(outputs.shape))

df3 = pd.read_csv('input_validation_data.csv')
testIn = df3.to_numpy()
df4 = pd.read_csv('output_validation_data.csv')
testOut = df4.to_numpy()

print ('The shape of the input validation is: ' + str(testIn.shape))
print ('The shape of the output validation is: ' + str(testOut.shape))

The shape of the input training is: (14910, 367)
The shape of the output training is: (14910, 4)
The shape of the input validation is: (149, 367)
The shape of the output validation is: (149, 4)


In [14]:
#Setting up a neural network. There are 367 inputs and 4 outputs. The layers can be altered as desired.
model = Sequential(
    [               
        tf.keras.Input(shape=(367,)),    #specify input size
        ### START CODE HERE ### 
        Dense(25, activation='relu'),
        Dense(100, activation='relu'),
        Dense(150, activation='relu'),
        Dense(200, activation='relu'),
        Dense(200, activation='relu'),
        Dense(150, activation='relu'),
        Dense(100, activation='relu'),
        Dense(4, activation='sigmoid')
        
        
        ### END CODE HERE ### 
    ], name = "my_model" 
)

In [15]:
#Printing the summary of the model
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 25)                9200      
                                                                 
 dense_24 (Dense)            (None, 100)               2600      
                                                                 
 dense_25 (Dense)            (None, 150)               15150     
                                                                 
 dense_26 (Dense)            (None, 200)               30200     
                                                                 
 dense_27 (Dense)            (None, 200)               40200     
                                                                 
 dense_28 (Dense)            (None, 150)               30150     
                                                                 
 dense_29 (Dense)            (None, 100)               151

In [16]:
#Training the model with 250 epochs. The legacy optimizer is used for Mac computers with M1/M2 chips.
model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.legacy.Adam(0.001),
)
model.fit(
    inputs,outputs,
    epochs=150
)

Epoch 1/150
466/466 [==============================] - 1s 880us/step - loss: 0.9687
Epoch 2/150
466/466 [==============================] - 1s 2ms/step - loss: 0.8475
Epoch 3/150
466/466 [==============================] - 1s 2ms/step - loss: 0.8132
Epoch 4/150
466/466 [==============================] - 0s 987us/step - loss: 0.7898
Epoch 5/150
466/466 [==============================] - 1s 1ms/step - loss: 0.7733
Epoch 6/150
466/466 [==============================] - 0s 989us/step - loss: 0.7575
Epoch 7/150
466/466 [==============================] - 0s 962us/step - loss: 0.7382
Epoch 8/150
466/466 [==============================] - 1s 1ms/step - loss: 0.7300
Epoch 9/150
466/466 [==============================] - 0s 920us/step - loss: 0.7183
Epoch 10/150
466/466 [==============================] - 0s 979us/step - loss: 0.7079
Epoch 11/150
466/466 [==============================] - 0s 979us/step - loss: 0.7001
Epoch 12/150
466/466 [==============================] - 0s 926us/step - loss: 0.68

466/466 [==============================] - 0s 1ms/step - loss: 0.3425
Epoch 99/150
466/466 [==============================] - 0s 942us/step - loss: 0.3389
Epoch 100/150
466/466 [==============================] - 0s 950us/step - loss: 0.3395
Epoch 101/150
466/466 [==============================] - 1s 1ms/step - loss: 0.3420
Epoch 102/150
466/466 [==============================] - 0s 998us/step - loss: 0.3352
Epoch 103/150
466/466 [==============================] - 0s 957us/step - loss: 0.3432
Epoch 104/150
466/466 [==============================] - 0s 966us/step - loss: 0.3269
Epoch 105/150
466/466 [==============================] - 0s 960us/step - loss: 0.3472
Epoch 106/150
466/466 [==============================] - 0s 976us/step - loss: 0.3407
Epoch 107/150
466/466 [==============================] - 0s 979us/step - loss: 0.3341
Epoch 108/150
466/466 [==============================] - 1s 1ms/step - loss: 0.3378
Epoch 109/150
466/466 [==============================] - 0s 938us/step - lo

In [9]:
#Function to encode an input for the neural network
    #p: seat
    #p1: players left
    #bet: highest bet
    #h1: first hole card number
    #hs1: first hole card suit
    #h2: second hole card number
    #hs2: second hole card suit
    #c1: first community card number
    #cs1: first community card suit
    #c2: second community card number
    #cs2: second community card suit
    #c3: third community card number
    #cs3: third community card suit
    #c4: fourth community card number
    #cs4: fourth community card suit
    #c5: fifth community card number
    #cs5: fifth community card suit
    #Card numbers are integers. Ace is low (1)
    #Card suits are as follows: spades is 1, clubs is 2, diamonds is 3, hearts is 4

def encode_input(p, pl, bet, h1, hs1, h2, hs2, c1, cs1, c2, cs2, c3, cs3, c4, cs4, c5, cs5):
    inp = [0.0] * 367
    inp[0] = p
    inp[1] = pl
    inp[2] = math.log(bet, 5)
    inp[2 + h1 + 13 * (hs1-1)] = 1
    inp[2 + 52 + h2 + 13 * (hs2-1)] = 1
    if c1 != 0:
        inp[2 + 52 * 2 + c1 + 13 * (cs1-1)] = 1
    if c2 != 0:
        inp[2 + 52 * 3 + c2 + 13 * (cs2-1)] = 1
    if c3 != 0:
        inp[2 + 52 * 4 + c3 + 13 * (cs3-1)] = 1
    if c4 != 0:
        inp[2 + 52 * 5 + c4 + 13 * (cs4-1)] = 1
    if c5 != 0:
        inp[2 + 52 * 6 + c5 + 13 * (cs5-1)] = 1
    return inp

In [10]:
#Testing the model and the input encoding function using an input from the validation data
print("Testing with the following inputs: Seat 3, 2 players left, 100 in pot, Kc Ah hole, 6c Qd 2s Ad community after the turn")
x = np.array(encode_input(3, 2, 100, 13, 2, 1, 4, 6, 2, 12, 3, 2, 1, 1, 3, 0, 0))
prediction = model.predict(x.reshape(1,367))
print(f" predicting with the encoded input: {prediction}")
prediction = model.predict(testIn[143].reshape(1,367))
print(f" predicting from the validation data: {prediction}")
print(f" AI's action from the validation data: {testOut[143]}")


Testing with the following inputs: Seat 3, 2 players left, 100 in pot, Kc Ah hole, 6c Qd 2s Ad community after the turn
1/1 [==============================] - 0s 78ms/step
 predicting with the encoded input: [[6.1985033e-05 8.3821625e-01 3.3009797e-05 1.7276736e-01]]
1/1 [==============================] - 0s 8ms/step
 predicting from the validation data: [[6.1985033e-05 8.3821625e-01 3.3009797e-05 1.7276736e-01]]
 AI's action from the validation data: [0. 0. 0. 1.]


In [6]:
def softmax(x):
    return np.exp(x)/sum(np.exp(x))

In [17]:
test_predictions = model.predict(testIn)

correct = 0
for i, pred in enumerate(test_predictions):
    if np.argmax(pred) == np.argmax(testOut[i]):
        correct += 1
    else:
        print(f"Predicted: {softmax(pred)}, actual: {testOut[i]}")

print(f"Correct: {correct}  Incorrect: {len(test_predictions) - correct} accuracy: {correct / len(test_predictions)}")

5/5 [==============================] - 0s 612us/step
Predicted: [0.19492264 0.17724843 0.17719325 0.45063567], actual: [1. 0. 0. 0.]
Predicted: [0.16832128 0.32185864 0.31217393 0.19764611], actual: [1. 0. 0. 0.]
Predicted: [0.4174074  0.16138437 0.16127643 0.2599318 ], actual: [0. 0. 0. 1.]
Predicted: [0.22823371 0.16768038 0.3915777  0.21250817], actual: [1. 0. 0. 0.]
Predicted: [0.1500457  0.37211847 0.20750858 0.2703272 ], actual: [0. 0. 1. 0.]
Predicted: [0.16168189 0.2887658  0.31300905 0.23654327], actual: [0. 0. 0. 1.]
Predicted: [0.14728673 0.3398785  0.34333256 0.1695022 ], actual: [0. 0. 0. 1.]
Predicted: [0.14817901 0.29190597 0.28113192 0.27878308], actual: [0. 0. 1. 0.]
Predicted: [0.47453877 0.17485452 0.17515466 0.17545208], actual: [0. 0. 1. 0.]
Predicted: [0.17768806 0.3341607  0.3192436  0.16890769], actual: [0. 0. 1. 0.]
Predicted: [0.46014312 0.18540868 0.17734614 0.17710206], actual: [0. 0. 0. 1.]
Predicted: [0.39930242 0.16363269 0.1629016  0.2741633 ], actual: [

In [117]:
#Place any set of input here to receive the model output!
x = np.array(encode_input(3, 4, 200, 12, 2, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))
prediction = model.predict(x.reshape(1,367))
print(f" predicting with the encoded input: {prediction}")

1/1 [==============================] - 0s 9ms/step
 predicting with the encoded input: [[9.99859929e-01 1.69179457e-05 2.27208584e-05 1.15462186e-04]]
